<a href="https://colab.research.google.com/github/fralfaro/clei2025-llm/blob/main/docs/dia1/notebooks/structured-outputs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Extra: Structured Outputs con OpenAI y Langchain

Para mayor información: https://python.langchain.com/docs/concepts/structured_outputs/

In [1]:
# %pip install pydantic python-dotenv langchain langchain-openai langchain-community

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

---

## OpenAI

In [4]:
from pydantic import BaseModel
from openai import OpenAI

client = OpenAI()

class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

completion = client.chat.completions.parse(
    model="gpt-4.1-nano",
    messages=[
        {"role": "system", "content": "Extract the event information."},
        {"role": "user", "content": "Alice and Bob are going to a science fair on Friday."},
    ],
    response_format=CalendarEvent,
)

event = completion.choices[0].message.parsed
print(event)


name='Science Fair' date='Friday' participants=['Alice', 'Bob']


---

## Langchain (sobre OpenAI)

In [2]:
from pydantic import BaseModel

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage

model = ChatOpenAI(model="gpt-4.1-nano", temperature=0)

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="Extract the event information."),
        HumanMessage(content="Alice and Bob are going to a science fair on Friday."),
    ]
)

class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

# Bind schema to model
model_with_tools = model.with_structured_output(CalendarEvent)

# Invoke the model to produce structured output that matches the schema
response = model_with_tools.invoke(chat_template.format_messages())


In [3]:
print(response)

name='Science Fair' date='Friday' participants=['Alice', 'Bob']


---